In [11]:
import json
import os
from dotenv import load_dotenv, find_dotenv
from fhir_kindling import upload_bundle, query_server, FhirServer
from fhir_kindling.serde import load_bundle, validate_bundle



In [2]:
%load_ext autoreload
%autoreload 2

In [9]:
load_dotenv(find_dotenv())

username = os.getenv("FHIR_USER")
password = os.getenv("FHIR_PW")
token = os.getenv("FHIR_TOKEN")

token


'eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJ5VmMwcnVQZjdrMDgxN2JWMWF0ZFoycWpJUUFqYnR3RUpiZklvZ3k3aElzIn0.eyJleHAiOjE2MzAwOTAxNDUsImlhdCI6MTYzMDA1NDE0NSwianRpIjoiMmFhMTYwZmQtYTQxMy00NDhlLWEwMzgtZmExOTgxMTZmODMxIiwiaXNzIjoiaHR0cHM6Ly9rZXljbG9hay1waHQudGFkYTVoaS5uZXQvYXV0aC9yZWFsbXMvYmxhemUiLCJzdWIiOiI3MmE3ZjM3ZS1iMzNmLTQ5MDgtOWFkOS0zM2JlMGQ0YzE2MjAiLCJ0eXAiOiJCZWFyZXIiLCJhenAiOiJhY2NvdW50IiwiYWNyIjoiMSIsInJlc291cmNlX2FjY2VzcyI6eyJhY2NvdW50Ijp7InJvbGVzIjpbIm1hbmFnZS1hY2NvdW50Iiwidmlldy1hcHBsaWNhdGlvbnMiLCJ2aWV3LWNvbnNlbnQiLCJtYW5hZ2UtYWNjb3VudC1saW5rcyIsImRlbGV0ZS1hY2NvdW50IiwibWFuYWdlLWNvbnNlbnQiLCJ2aWV3LXByb2ZpbGUiXX19LCJzY29wZSI6ImVtYWlsIHByb2ZpbGUiLCJjbGllbnRJZCI6ImFjY291bnQiLCJjbGllbnRIb3N0IjoiMTkyLjE2OC4wLjEiLCJlbWFpbF92ZXJpZmllZCI6ZmFsc2UsInByZWZlcnJlZF91c2VybmFtZSI6InNlcnZpY2UtYWNjb3VudC1hY2NvdW50IiwiY2xpZW50QWRkcmVzcyI6IjE5Mi4xNjguMC4xIn0.V3-ec6_hvzRCFcAoF2cqxqT6mPNOTmG7hJX7NLqYnrQ_qNomk0xb9rSkphV1ErkT2EYY2tp25EoCCAV4ErzF6mcyqV--jIhjz41wye-iZL8l1M2eD3_ylpkWTH7nlTzoXiIdGqh

In [8]:
polar_bundle_1 = "./polar/bundles/POLAR_Testdaten_UKB.json"

bundle_json = json.load(open(polar_bundle_1))
bundle = validate_bundle(bundle_json, drop_invalid_entries=True, show_invalid_entries=True)


2482


In [11]:
upload_bundle(bundle, fhir_api_url=os.getenv("BLAZE_API_URL"), token=token)



{'id': 'C7AT5ENIPB3G2NG2',
 'type': 'transaction-response',
 'entry': [{'response': {'status': '201',
    'etag': 'W/"6"',
    'lastModified': '2021-09-23T18:26:55.558Z',
    'location': 'https://blaze-fhir.personalhealthtrain.de/fhir/Encounter/C7AT5EKUH7YLTZ2I/_history/6'}},
  {'response': {'status': '201',
    'etag': 'W/"6"',
    'lastModified': '2021-09-23T18:26:55.558Z',
    'location': 'https://blaze-fhir.personalhealthtrain.de/fhir/Encounter/C7AT5EKUH7YLTZ2J/_history/6'}},
  {'response': {'status': '201',
    'etag': 'W/"6"',
    'lastModified': '2021-09-23T18:26:55.558Z',
    'location': 'https://blaze-fhir.personalhealthtrain.de/fhir/Encounter/C7AT5EKUH7YLTZ2K/_history/6'}},
  {'response': {'status': '201',
    'etag': 'W/"6"',
    'lastModified': '2021-09-23T18:26:55.558Z',
    'location': 'https://blaze-fhir.personalhealthtrain.de/fhir/Encounter/C7AT5EKUH7YLTZ2L/_history/6'}},
  {'response': {'status': '201',
    'etag': 'W/"6"',
    'lastModified': '2021-09-23T18:26:55.558Z

In [10]:

query_df = query_server(resource="Encounter",
                 fhir_server_url=os.getenv("BLAZE_API_URL"),
                 token=token,
                 fhir_server_type="blaze",
                 out_format="df")
query_df

HTTPError: 401 Client Error: Unauthorized for url: https://blaze-fhir.personalhealthtrain.de/fhir/Encounter?_count=1000

In [19]:
server = FhirServer(api_address=os.getenv("BLAZE_API_URL"), token=os.getenv("FHIR_TOKEN"))
server.meta_data().dict()["rest"]


[{'interaction': [{'code': 'search-system'},
   {'code': 'transaction'},
   {'code': 'batch'},
   {'code': 'history-system'}],
  'mode': 'server',
  'resource': [{'conditionalCreate': True,
    'conditionalDelete': 'not-supported',
    'conditionalRead': 'not-supported',
    'conditionalUpdate': False,
    'interaction': [{'code': 'read'},
     {'code': 'vread'},
     {'code': 'update'},
     {'code': 'delete'},
     {'code': 'history-instance'},
     {'code': 'history-type'},
     {'code': 'create'},
     {'code': 'search-type'}],
    'readHistory': True,
    'referencePolicy': ['literal', 'enforced', 'local'],
    'searchParam': [{'definition': 'http://hl7.org/fhir/SearchParameter/Resource-id',
      'name': '_id',
      'type': 'token'},
     {'definition': 'http://hl7.org/fhir/SearchParameter/Resource-lastUpdated',
      'name': '_lastUpdated',
      'type': 'date'},
     {'definition': 'http://hl7.org/fhir/SearchParameter/Resource-profile',
      'name': '_profile',
      'type': 